Import necessary libraries

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from keras import regularizers
from tensorflow import keras
from keras.layers import Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
import nltk
from nltk.corpus import stopwords
import string
from google.colab import drive
import math
import re # module to remove non-alphanumeric chars

Load and view data

## 1. Here imported the Dr. Hibo Je Smaple Data Set

In [ ]:
# Load data
drive.mount('/content/drive')
df_1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ANN/Group Project 2/RateMyProfessor_Sample data.csv")
print(df_1.head(3))

Mounted at /content/drive
   professor_name                                 school_name  \
0  Leslie  Looney  University Of Illinois at Urbana-Champaign   
1  Leslie  Looney  University Of Illinois at Urbana-Champaign   
2  Leslie  Looney  University Of Illinois at Urbana-Champaign   

        department_name                    local_name state_name  \
0  Astronomy department   Champaign\xe2\x80\x93Urbana         IL   
1  Astronomy department   Champaign\xe2\x80\x93Urbana         IL   
2  Astronomy department   Champaign\xe2\x80\x93Urbana         IL   

   year_since_first_review  star_rating take_again  diff_index  \
0                     11.0          4.7        NaN         2.0   
1                     11.0          4.7        NaN         2.0   
2                     11.0          4.7        NaN         2.0   

                                       tag_professor  ...  lots_of_homework  \
0  Hilarious (2)  GROUP PROJECTS (2)  Gives good ...  ...                 0   
1  Hilarious (2) 

In [ ]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 51 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   professor_name            20000 non-null  object 
 1   school_name               20000 non-null  object 
 2   department_name           20000 non-null  object 
 3   local_name                20000 non-null  object 
 4   state_name                20000 non-null  object 
 5   year_since_first_review   20000 non-null  float64
 6   star_rating               20000 non-null  float64
 7   take_again                2998 non-null   object 
 8   diff_index                20000 non-null  float64
 9   tag_professor             11093 non-null  object 
 10  num_student               20000 non-null  float64
 11  post_date                 19995 non-null  object 
 12  name_onlines              20000 non-null  object 
 13  name_not_onlines          19995 non-null  object 
 14  studen

In [ ]:
# Check types of the features
df_1.dtypes

professor_name               object
school_name                  object
department_name              object
local_name                   object
state_name                   object
year_since_first_review     float64
star_rating                 float64
take_again                   object
diff_index                  float64
tag_professor                object
num_student                 float64
post_date                    object
name_onlines                 object
name_not_onlines             object
student_star                float64
student_difficult           float64
attence                      object
for_credits                  object
would_take_agains            object
grades                       object
help_useful                 float64
help_not_useful             float64
comments                     object
word_comment                float64
gender                       object
race                         object
asian                       float64
hispanic                    

Extract data and pre-process

In [ ]:
# Extract comments and the 2 labels
df_1 = df_1.iloc[:, [22, 14, 15]]
df_1.head(3)

,comments,student_star,student_difficult
0,"This class is hard, but its a two-in-one gen-e...",5.0,3.0
1,Definitely going to choose Prof. Looney\'s cla...,5.0,2.0
2,I overall enjoyed this class because the assig...,4.0,3.0


2. Here Imported the smaller Data Set in json format from Kaggle

In [ ]:
df_2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ANN/Group Project 2/out.csv")
df_2 = df_2.rename(columns={'Comment': 'comments', 'Quality': 'student_star', 'Difficulty':'student_difficult'})

In [ ]:
df_2 = df_2.iloc[:, [7, 1, 2]]
df_2.head(3)

,comments,student_star,student_difficult
0,Professor Nichols is super nice and very whole...,5.0,2.0
1,"Prof Humes is really knowledgeable, but her le...",4.0,4.0
2,If you fall behind it's almost impossible to c...,3.0,4.0


In [ ]:
df = pd.concat([df_1, df_2])

3. Here is the Web Scapping Data stored in "reviews.txt"

*   I have used the Python Selenium to do the web scrapping for list of professors
*   Python Selenium did almost ran for 6 hours for collecting data from 300 professors
*   Below shared the smaple code for that
*   Below is sample code but i have executed in my loal to gather more data





In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains


browser = webdriver.Chrome(executable_path='/content/drive/MyDrive/Colab Notebooks/ANN/Group Project 2/web scrapping/chromedriver.exe')
Load_xpath = "//button[contains(text(),'Load More Ratings')]"
url = []
js_code = "arguments[0].scrollIntoView({'block':'center','inline':'center'});"
file = open("/content/drive/MyDrive/Colab Notebooks/ANN/Group Project 2/web scrapping/Python Programs/reviews.txt","a")
text_num_review_g = 0

with open('/content/drive/MyDrive/Colab Notebooks/ANN/Group Project 2/web scrapping/Python Programs/newdoc.txt', 'r') as file_url:
    for line in file_url:
        # Process the line
        url.append(line.strip()) # Example: Print the line after removing leading/trailing whitespaces

def check_exists_by_xpath(xpath):
    try:
        browser.find_element(By.XPATH, xpath)
    except NoSuchElementException:
        return False
    return True

def proff_reviews(num):
    # df = pd.read_csv("out.csv")
    # df.columns = ['comments','student_star','student_difficult']
    # columns = "comments,student_star,student_difficult"
    # file.write(f"{columns}\n")
    if (check_exists_by_xpath("//a[@href='#ratingsList']")):
        num_review = browser.find_element(By.XPATH, "//a[@href='#ratingsList']")
        text_num_review = num_review.text
        text_num_review = text_num_review.split(" ")
        text_num_review = int(text_num_review[0])
        num = num + text_num_review


        while(check_exists_by_xpath(Load_xpath)):
            Load_element = browser.find_element(By.XPATH, "//button[contains(text(),'Load More Ratings')]")
            # actions = ActionChains(browser)
            # actions.move_to_element(Load_element).perform()
            # Execute the JS script
            browser.execute_script(js_code, Load_element)
            time.sleep(3)
            Load_element.click()
            time.sleep(3)

        comments = browser.find_elements(By.XPATH, "//div[@class='Comments__StyledComments-dzzyvm-0 gRjWel']")
        quality = browser.find_elements(By.XPATH, "//div[contains(text(),'Quality')]/following-sibling::div")
        difficulty = browser.find_elements(By.XPATH, "//div[contains(text(),'Difficulty')]/following-sibling::div")

        text_comments = []
        text_quality = []
        text_difficulty = []
        
        for i in comments:
            text_comments.append(i.text)
        
        for i in quality:
            text_quality.append(i.text)
        
        for i in difficulty:
            text_difficulty.append(i.text)

        if len(text_comments) == len(text_quality) and len(text_comments) == len(text_difficulty):
            for i in range(0,len(text_comments)):
                # new_row = "text_comments[i], text_quality[i], text_difficulty[i]"
                file.write(f"{text_comments[i]},{text_quality[i]},{text_difficulty[i]}\n")
    return num




for i in range(0,100):
    browser.get(url[i])
    text_num_review_g = proff_reviews(text_num_review_g)
    print(url[i])

print(text_num_review_g)
file.close()

browser.quit()

<ipython-input-13-7e10936c7132>:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path='/content/drive/MyDrive/Colab Notebooks/ANN/Group Project 2/web scrapping/chromedriver.exe')


WebDriverException: ignored

In [ ]:
df_3 = pd.DataFrame()

# add columns to the DataFrame
df_3['comments'] = []
df_3['student_star'] = []
df_3['student_difficult'] = []

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/ANN/Group Project 2/reviews.txt', 'r') as file:
    for line in file:
        line = line.strip()  # remove any leading or trailing whitespace
        parts = line.rsplit(',', maxsplit=2)  # split the line from the end with only 3 length
        # add data one row at a time
        df_3 = df_3.append({'comments': parts[0], 'student_star': int(float(parts[1])), 'student_difficult':int(float(parts[2]))}, ignore_index=True)

Streaming output truncated to the last 5000 lines.
<ipython-input-48-5eaf1253be99>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3 = df_3.append({'comments': parts[0], 'student_star': int(float(parts[1])), 'student_difficult':int(float(parts[2]))}, ignore_index=True)
<ipython-input-48-5eaf1253be99>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3 = df_3.append({'comments': parts[0], 'student_star': int(float(parts[1])), 'student_difficult':int(float(parts[2]))}, ignore_index=True)
<ipython-input-48-5eaf1253be99>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3 = df_3.append({'comments': parts[0], 'student_star': int(float(parts[1])), 'student_difficult':int(float(parts[2]))}, ignore_index=True)
<ipython-inpu

In [ ]:
df_3.dtypes

comments              object
student_star         float64
student_difficult    float64
dtype: object

In [ ]:
df = pd.concat([df, df_3])
df['student_star'] = df['student_star'].fillna(0)
df['student_difficult'] = df['student_difficult'].fillna(0)
df['student_star'] = df['student_star'].astype(int)
df['student_difficult'] = df['student_difficult'].astype(int)

In [ ]:
df = df.drop(df[df['student_star'] == 0].index)
df = df.drop(df[df['student_difficult'] == 0].index)

In [ ]:
df

,comments,student_star,student_difficult
0,"This class is hard, but its a two-in-one gen-e...",5,3
1,Definitely going to choose Prof. Looney\'s cla...,5,2
2,I overall enjoyed this class because the assig...,4,3
3,"Yes, it\'s possible to get an A but you\'ll de...",5,3
4,Professor Looney has great knowledge in Astron...,5,1
...,...,...,...
22143,"Great Professor, very helpful. Fun class, but ...",4,2
22144,"Very thorough in explaining things, makes lear...",5,2
22145,I really enjoy his teaching style he keeps cla...,5,3
22146,I had Mr. Dixon last semester. I now got to SM...,5,3


In [ ]:
# Find any NaN in the two labels
print(np.where(np.isnan(df['student_star'])))
print(np.where(np.isnan(df['student_difficult'])))

# Drop the observations that contain a NaN(s) as a label
df = df.dropna(subset=["student_star"])

(array([], dtype=int64),)
(array([], dtype=int64),)


In [ ]:
# Check if NaN were successfully removed
print(np.where(np.isnan(df['student_star'])))
print(np.where(np.isnan(df['student_difficult'])))

print("New shape", df.shape)

(array([], dtype=int64),)
(array([], dtype=int64),)
New shape (67583, 3)


In [ ]:
# Extract comments, student star rating, and student difficulty rating
x = df["comments"].astype(str).tolist()
y_star = df.iloc[:, 1].values
y_diff = df.iloc[:, 2].values

print(type(x), type(y_star), type(y_diff))
print(len(x), y_star.size, y_diff.size)

<class 'list'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
67583 67583 67583


In [ ]:
# Removes any character that is not a word or whitespace while lowercasing
cleaning_x = []
for i in x:
  i = re.sub(r'[^\w\s]', ' ', i)
  i = i.lower()
  cleaning_x.append(i)

# Remove \
cleaned_x = []
for j in cleaning_x:
  j = j.replace("\\", " ")
  cleaned_x.append(j)

# Remove stopwords if have time
# Get NLTK stopwords
nltk.download('stopwords')
cleanest_text = []
for q in cleaned_x:
  words = q.split()
  cleaned = []
  for w in words:
    if w not in set(stopwords.words('english')):
      cleaned.append(w)
  cleanest_text.append(' '.join(cleaned))

#cleanest_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
len(cleanest_text)

67583

In [ ]:
# Tokenization
tokens = tf.keras.preprocessing.text.Tokenizer(num_words = 20000)

# Index vocabulary, so can make sequence later
tokens.fit_on_texts(cleanest_text)

# Convert the student comments to a sequence of integers
integer_sequences = tokens.texts_to_sequences(cleanest_text)

# Take a look at one student comment in the form of an integer sequence
print(integer_sequences[69])

[119, 1103, 44, 3395, 1064, 158, 32, 21, 315, 47, 60, 154, 825, 378, 3221, 1, 1400, 24, 6400, 674, 1007, 172, 1116, 436, 2086, 1880, 1]


In [ ]:
# Padding the sequence to make them all the same lengths
# Note: Sequences must have same length to feed into NN

# Find and set max length from sequences to pad with zeroes
max_length = 0
for k in integer_sequences:
  if len(k) > max_length:
    max_length = len(k)
print("Max length:", max_length)

padded_sequences = tf.keras.utils.pad_sequences(integer_sequences,
                                                maxlen = max_length,
                                                padding = "post")
print(padded_sequences.shape)
print(padded_sequences[69])
print(integer_sequences[69])

Max length: 63
(67583, 63)
[ 119 1103   44 3395 1064  158   32   21  315   47   60  154  825  378
 3221    1 1400   24 6400  674 1007  172 1116  436 2086 1880    1    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0]
[119, 1103, 44, 3395, 1064, 158, 32, 21, 315, 47, 60, 154, 825, 378, 3221, 1, 1400, 24, 6400, 674, 1007, 172, 1116, 436, 2086, 1880, 1]


Load in Stanford's GloVe, which is a pre-trained word embedding application

In [ ]:
# Chollet's Style
embeddings_index = {}
with open("/content/drive/MyDrive/Colab Notebooks/ANN/Group Project 2/glove.6B.100d.txt") as f:
  for line in f:
    word, coefs = line.split(maxsplit = 1)
    coefs = np.fromstring(coefs, "f" ,sep = " ")
    embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))
#embeddings_index.keys()
#embeddings_index['is']

Found 400000 word vectors.


In [ ]:
# Preparing/Creating the GloVe word-embeddings matrix
embedding_dimensions = 100
word_index = tokens.word_index

embedding_matrix = np.zeros((len(word_index) + 1, embedding_dimensions))
for word, i in word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix.shape

(24233, 100)

In [ ]:
max_length

63

In [ ]:
len(word_index)

24232

In [ ]:
# Define the Embedding layer
embedding_layer = tf.keras.layers.Embedding(input_dim = len(word_index) + 1,
                                            output_dim = embedding_dimensions,
                                            input_length = max_length,
                                            weights = [embedding_matrix],
                                            embeddings_initializer = tf.keras.initializers.Constant(embedding_matrix),
                                            trainable = False,
                                            mask_zero=True)

In [ ]:
padded_sequences.shape

(67583, 63)

In [ ]:
padded_sequences[0].shape[0]

63

In [ ]:
# Use Embedding Layer to convert integer sequences to vector sequences
# skip for now, since i think it is being embedded already during training process due to the first embedding layer
#embedded_sequences = embedding_layer(padded_sequences)

In [ ]:
#embedded_sequences.shape

Splitting the data into 60% training, 20% validation, 20% testing while shuffling

In [ ]:
# Convert embedded_sequences from Tensor to np.array for splitting
# Note: Convert from np.array to Tensor after to feed to NN
x_np = padded_sequences
print(x_np.shape, type(x_np))
print(y_star.shape, y_diff.shape)

(67583, 63) <class 'numpy.ndarray'>
(67583,) (67583,)


In [ ]:
x_tv, x_test, ys_tv, ys_test, yd_tv, yd_test=train_test_split(
    x_np,y_star,y_diff,test_size=0.2, shuffle=True)
x_train,x_val,ys_train,ys_val,yd_train,yd_val=train_test_split(
    x_tv,ys_tv,yd_tv,test_size=0.25, shuffle=True)
print(x_train.shape, x_val.shape, x_test.shape)
print(ys_train.shape, ys_val.shape, ys_test.shape)
print(yd_train.shape, yd_val.shape, yd_test.shape)

(40549, 63) (13517, 63) (13517, 63)
(40549,) (13517,) (13517,)
(40549,) (13517,) (13517,)


In [ ]:
# Convert from np.array to Tensor after to feed to NN
x_train = tf.convert_to_tensor(x_train)
x_val = tf.convert_to_tensor(x_val)
x_test = tf.convert_to_tensor(x_test)
ys_train = tf.convert_to_tensor(ys_train)
ys_val = tf.convert_to_tensor(ys_val)
ys_test = tf.convert_to_tensor(ys_test)
yd_train = tf.convert_to_tensor(yd_train)
yd_val = tf.convert_to_tensor(yd_val)
yd_test = tf.convert_to_tensor(yd_test)

In [ ]:
x_train.shape

TensorShape([40549, 63])

In [ ]:
ys_train = to_categorical(ys_train, num_classes=9)
ys_val = to_categorical(ys_val, num_classes=9)
ys_test = to_categorical(ys_test, num_classes=9)
yd_train = to_categorical(yd_train, num_classes=9)
yd_val = to_categorical(yd_val, num_classes=9)
yd_test = to_categorical(yd_test, num_classes=9)

The models and layers

In [ ]:
x_train[0]

<tf.Tensor: shape=(63,), dtype=int32, numpy=
array([ 1358,    14,  4088,    67,   252,     1,   145,    48,    63,
        7556,   864, 13841,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0],
      dtype=int32)>

In [ ]:
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

model = keras.Sequential()
model.add(embedding_layer)
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.5))
model.add(Dense(9, activation='softmax'))

model.summary()
from tensorflow.keras.optimizers import Adam
#optimizer = Adam(learning_rate=0.03) # default 0.001
model.compile(loss='categorical_crossentropy', optimizer="rmsprop", metrics=['accuracy'])

model.fit(x_train, ys_train, validation_data=(x_val, ys_val),
           epochs=100, batch_size=32)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 63, 100)           2423300   
                                                                 
 bidirectional_2 (Bidirectio  (None, 64)               34048     
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 9)                 585       
                                                                 
Total params: 2,457,933
Trainable params: 34,633
Non-trainable params: 2,423,300
_________________________________________________________________
Epoch 1/100
1268/1268 [==============================] - 121s 88ms/step - loss: 1.2880 - accuracy: 0.4597

In [ ]:
model = keras.Sequential()
model.add(embedding_layer)
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.5))
model.add(Dense(9, activation='softmax'))

model.summary()
from tensorflow.keras.optimizers import Adam
#optimizer = Adam(learning_rate=0.03) # default 0.001
model.compile(loss='categorical_crossentropy', optimizer="rmsprop", metrics=['accuracy'])

model.fit(x_train, yd_train, validation_data=(x_val, yd_val),
           epochs=100, batch_size=32)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 63, 100)           2423300   
                                                                 
 bidirectional_3 (Bidirectio  (None, 64)               34048     
 nal)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 9)                 585       
                                                                 
Total params: 2,457,933
Trainable params: 34,633
Non-trainable params: 2,423,300
_________________________________________________________________
Epoch 1/100
1268/1268 [==============================] - 110s 80ms/step - loss: 1.5345 - accuracy: 0.3091